# Test Pix2Struct model on Synthetic Bootstrap dataset (mini version)

## Setup Envirnoment

In [2]:
pip install transformers==4.36.2


Defaulting to user installation because normal site-packages is not writeable
  Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0

[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: /opt/software-current/2023.06/x86_64/generic/software/Python/3.11.3-GCCcore-12.3.0/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import necessary libraries

In [1]:
import os
import zipfile
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import re
from transformers import Pix2StructForConditionalGeneration, AutoProcessor
import torch
from torch.nn import functional as F
from pathlib import Path
from nltk import edit_distance
import numpy as np
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
from torch.utils.data import random_split
import random

/mnt/home/seyeon/.local/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/mnt/home/seyeon/.local/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


## Define variables and parameters

In [2]:
# FOLDER_CHECKPOINTS = 'checkpoints'
DATASET_NAME = 'pix2code/'
# ZIP_NAME = DATASET_NAME + '.zip'
DESTINATION_FOLDER= 'data/'
DATASET_FOLDER = DESTINATION_FOLDER + DATASET_NAME
HTML_FILES_FOLDER = DATASET_FOLDER + "html/"
OUTPUT_FOLDER = 'text_pix2code/'

EXPERIMENT_NAME = "pix2code"

MAX_SENTENCE_LEN = 4096

CHUNK_LENGTH = 1024
CONTEXT_OVERLAP_LENGTH = 256

MAX_PATCHES = 1024

DEBUG = False
VERBOSE = True

BATCH_SIZE = 10

TRAIN_SET_PERCENTAGE = 0.88
VALID_SET_PERCENTAGE = 0.02 # Use 20 for validation
# TEST_SET_PERCENTAGE is 1 - TRAIN_SET_PERCENTAGE - VALID_SET_PERCENTAGE # Use 100 for test

RANDOM_SEED = 123

LOAD_FROM_CHECKPOINT = True
LAST_CHECKPOINT_NAME = "pix2code_epoch[29]_bleu[0.82].pth"

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
MAX_N_CHUNKS_PER_SENTENCE = 1 + (MAX_SENTENCE_LEN - CHUNK_LENGTH) // (CHUNK_LENGTH - CONTEXT_OVERLAP_LENGTH)
print("MAX_N_CHUNKS_PER_SENTENCE", MAX_N_CHUNKS_PER_SENTENCE)

MAX_N_CHUNKS_PER_SENTENCE 5



## Load Synthetic Bootstrap Dataset (mini version)

## Load Model and Processor

In [5]:
repo_id = "google/pix2struct-base"

processor = AutoProcessor.from_pretrained(repo_id)
model = Pix2StructForConditionalGeneration.from_pretrained(repo_id, is_encoder_decoder=True)

/mnt/home/seyeon/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/231 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.61k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/851k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

## Create Dataset class

### Preprocessing functions

In [6]:
def round_floats_in_text(text, precision=0):
    # match float numbers with 2 or more decimal places in the text
    pattern = r"\b\d+\.\d{2,}\b"

    def replace(match):
        float_number = float(match.group())
        return f"{float_number:.{precision}f}"

    text = re.sub(pattern, replace, text)
    return text

In [7]:
def remove_html_comments(text):
    # match html comments
    pattern = r"<!--.*?-->"

    text = re.sub(pattern, '', text, flags=re.DOTALL)
    return text

In [8]:
def preprocess_html_file(html_text):
    text_cleaned = html_text.replace('\n', ' ')
    text_cleaned_without_multiple_spaces = re.sub(r'\s+', ' ', text_cleaned)
    text_without_comments = remove_html_comments(text_cleaned_without_multiple_spaces)
    text_without_long_floats = round_floats_in_text(text_without_comments)
    return text_without_long_floats

### Find max sentence length and new unknown tokens

In [9]:
# Find max length
all_paths = os.listdir(HTML_FILES_FOLDER)

In [10]:
print(HTML_FILES_FOLDER)

data/pix2code/html/


In [11]:
len(all_paths)

1742

In [12]:
all_paths = all_paths[:10000]

In [13]:
# # Find max length
# max_length = 0

# # Read text files and add new tokens to dictionary
# tokens_to_add = set()

# for html_file_path in all_paths:
#     file_path = os.path.join(HTML_FILES_FOLDER, html_file_path)
    
#     if os.path.isdir(file_path):
#         continue  # Skip directories
    
#     with open(file_path, "r") as reader:
#         splitted_text = processor.tokenizer(preprocess_html_file(reader.read())).tokens()
#         tokens_to_add = tokens_to_add.union(set(splitted_text))


    
#     # with open(HTML_FILES_FOLDER + html_file_path, "r") as reader:


#     #     splitted_text = processor.tokenizer(preprocess_html_file(reader.read())).tokens()
#     #     tokens_to_add = tokens_to_add.union(set(splitted_text))

#     # Check if the current sentence has the largest number of tokens
#     if len(splitted_text) > max_length:
#         max_length = len(splitted_text)

# print(f"Max sentence length = {max_length}")

# newly_added_num = processor.tokenizer.add_tokens(list(tokens_to_add))
# print(f"Number of new tokens = {newly_added_num}")

# # Resize the model's token embeddings if there are new tokens
# if newly_added_num > 0:
#     model.decoder.resize_token_embeddings(len(processor.tokenizer))


import chardet

# Find max length
max_length = 0
tokens_to_add = set()

for html_file_path in all_paths:
    file_path = os.path.join(HTML_FILES_FOLDER, html_file_path)
    
    if os.path.isdir(file_path):
        continue  # Skip directories
    
    # Detect encoding first
    with open(file_path, 'rb') as raw_reader:
        raw_data = raw_reader.read()
        encoding = chardet.detect(raw_data)['encoding']
    
    if encoding is None:
        print(f"⚠️ Skipping file due to unknown encoding: {file_path}")
        continue

    try:
        decoded_text = raw_data.decode(encoding)
    except Exception as e:
        print(f"⚠️ Skipping file {file_path} due to decoding error: {e}")
        continue

    try:
        splitted_text = processor.tokenizer(preprocess_html_file(decoded_text)).tokens()
        tokens_to_add.update(splitted_text)

        if len(splitted_text) > max_length:
            max_length = len(splitted_text)
    except Exception as e:
        print(f"⚠️ Tokenization error in {file_path}: {e}")
        continue

print(f"Max sentence length = {max_length}")

newly_added_num = processor.tokenizer.add_tokens(list(tokens_to_add))
print(f"Number of new tokens = {newly_added_num}")

if newly_added_num > 0:
    model.decoder.resize_token_embeddings(len(processor.tokenizer))


Max sentence length = 1251
Number of new tokens = 7749


### Split files into training - validation - test sets

In [14]:
random.seed(RANDOM_SEED)

# Use the same seed, so that parts remain the same
random.shuffle(all_paths)

train_len = int(TRAIN_SET_PERCENTAGE * len(all_paths))
valid_len = int(VALID_SET_PERCENTAGE * len(all_paths))

train_paths = all_paths[:train_len]
valid_paths = all_paths[train_len:train_len+valid_len]
test_paths = all_paths[train_len+valid_len:]

print(f"TRAIN_SET size = {len(train_paths)}")
print(f"VALID_SET size = {len(valid_paths)}")
print(f"TEST_SET size = {len(test_paths)}")

TRAIN_SET size = 1532
VALID_SET size = 34
TEST_SET size = 176


In [15]:
class SythBootstrapDataset(Dataset):
    def __init__(self, root_dir, transform, text_files_paths):

        self.root_dir = root_dir
        self.transform = transform
        self.text_files_paths = text_files_paths

        self.max_patches = MAX_PATCHES
        self.max_length = MAX_SENTENCE_LEN
        self.ignore_id = -100

        self.encodings = []

        for text_file in tqdm(text_files_paths):
            image_file = text_file.replace('.html', '.png')

            # Directly process the text files, and save them in the ram
            # Do the same also for images, if there is enough space in memory
            text_file_path = os.path.join(root_dir + "html/", text_file)
            image_file_path = os.path.join(root_dir + "images/", image_file)
            # print(image_file_path)
            # Load image
            image = Image.open(image_file_path).convert('RGB')

            if DEBUG:
                image.show()

            if self.transform:
                image = self.transform(image)

            encoding = processor(images=image, max_patches=self.max_patches, return_tensors="pt")
            encoding = {k:v.squeeze() for k,v in encoding.items()}

            # Load text
            with open(text_file_path, 'r') as f:
                text = f.read()
                text_cleaned = preprocess_html_file(text)

            if DEBUG:
              print("text:")
              print(text)
              print("\n\n\ntext_cleaned:")
              print(text_cleaned)

            input_ids = processor.tokenizer(
                text_cleaned,
                max_length=self.max_length,
                padding="max_length",
                truncation=True,
                return_tensors="pt",
            ).input_ids

            labels = input_ids.squeeze().clone()
            labels[labels == processor.tokenizer.pad_token_id] = self.ignore_id  # model doesn't need to predict pad token

            encoding["labels"] = labels.to(torch.int32)

            # For each sample save directly the encoding of both text and image
            self.encodings.append(encoding)

    def __len__(self):
        return len(self.encodings)

    def __getitem__(self, idx):
        return self.encodings[idx], self.text_files_paths[idx].replace(".html", "")

In [16]:
# Transformations for the image
transform = transforms.Compose([
    transforms.ToTensor(),  # convert PIL Image to PyTorch Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # normalize for pretrained models
])

# Instantiate the CustomDataset
test_dataset = SythBootstrapDataset(DATASET_FOLDER, transform, test_paths)

# Use DataLoader for batching and shuffling
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

100%|██████████| 176/176 [00:10<00:00, 16.12it/s]


In [17]:
print(f"test_dataloader size = {len(test_dataloader)}")

test_dataloader size = 18


In [18]:
batch = next(iter(test_dataloader))

In [19]:
encoding, text_file_paths = batch

In [20]:
print(len(encoding))

3


In [21]:
print(text_file_paths)

('B9261BA5-39AE-4261-9F43-29F25695C821', 'EA7488EC-E228-4630-9ACE-5875C4A1CC18', '06265836-7197-4CF0-9E9F-B7360B20B442', '6448D324-7312-427A-8EFE-277222EFCB71', '7903864F-8E16-4DB1-8950-7E94EA061DA3', 'EDF8AAF7-1813-4557-B1B1-2F12FEDA7E39', 'A3BDB090-DFB9-4191-89B3-4ACF951532CE', 'DC0ADA53-7AD5-4B1F-8937-EDA7DE2CADE0', '08705928-17E8-4AB8-BF2E-AE9F11BD26A2', '89BAB1BC-EEC1-4275-BB7B-F72948A818CA')


### Main Testing function

In [22]:
START_TOKEN_ID = PAD_TOKEN_ID = processor.tokenizer.pad_token_id

In [36]:
def testing_loop(testing_dataloader, model, processor, config, description):
    model.eval()
    bleu_scores = []
    ed_scores = []

    with torch.no_grad():
        test_loop = tqdm(enumerate(testing_dataloader), total=len(testing_dataloader), desc=description)
        for i, batch in test_loop:
            encoding, text_file_paths = batch
            encoding = move_to_device(encoding)
            labels, flattened_patches, attention_mask = encoding["labels"], encoding["flattened_patches"], encoding["attention_mask"]

            # Initialize total_outputs with zeros
            total_outputs = None
            context_from_last = None

            # Initialize a mask to track which sentences are finished
            finished_sentences_mask = torch.zeros(flattened_patches.size(0), dtype=torch.bool, device=flattened_patches.device)

            for iteration in range(MAX_N_CHUNKS_PER_SENTENCE):

                generate_args = {
                    "flattened_patches": flattened_patches[~finished_sentences_mask],
                    "attention_mask": attention_mask[~finished_sentences_mask],
                    "max_new_tokens": CHUNK_LENGTH - (CONTEXT_OVERLAP_LENGTH if iteration else 0),
                }

                if iteration and context_from_last is not None:
                    generate_args["decoder_input_ids"] = context_from_last[~finished_sentences_mask]

                outputs = model.generate(**generate_args)

                # Remove context overlap only from the second iteration onwards
                new_chunks = outputs if iteration == 0 else outputs[:, CONTEXT_OVERLAP_LENGTH:]

                if iteration == 0:
                    total_outputs = new_chunks
                else:
                    # Update total_outputs by concatenating new chunks
                    new_chunks_with_padding_chunks = torch.full((flattened_patches.shape[0], new_chunks.shape[1]), PAD_TOKEN_ID, dtype=new_chunks.dtype, device=new_chunks.device)
                    new_chunks_with_padding_chunks[~finished_sentences_mask] = new_chunks
                    total_outputs = torch.cat((total_outputs, new_chunks_with_padding_chunks), dim=1)

                # Update the finished_sentences_mask
                finished_sentences_mask[~finished_sentences_mask] |= (outputs == processor.tokenizer.eos_token_id).any(dim=1)

                # If all sentences are finished, exit the loop
                if finished_sentences_mask.all():
                    break

                if outputs.shape[1] < CHUNK_LENGTH:
                    print("ERROR: !! should have already exited because all sentences reached the end!!")

                # -1 because it will put in front a START_TOKEN automatically
                context_from_last = total_outputs[:, -(CONTEXT_OVERLAP_LENGTH-1):]

            predictions = processor.tokenizer.batch_decode(total_outputs, skip_special_tokens=True)

            labels[labels == -100] = 0
            answers = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)

            for pred, answer, text_file_path in zip(predictions, answers, text_file_paths):
                with open(f"{OUTPUT_FOLDER}/{text_file_path}_pred.txt", "w", encoding="utf-8") as f:
                    print(pred, file=f)

                with open(f"{OUTPUT_FOLDER}/{text_file_path}_answer.txt", "w", encoding="utf-8") as f:
                    print(answer, file=f)
    return

In [37]:
config = {
          "verbose": VERBOSE,
}

In [38]:
def validate_config(config):
    # Check required keys
    required_keys = [
        "verbose"
    ]
    for key in required_keys:
        if key not in config:
            raise ValueError(f"Key '{key}' must be present in the configuration.")

    # Check that values are in expected ranges
    if not isinstance(config["verbose"], bool):
        raise ValueError("verbose must be a boolean value.")

In [39]:
validate_config(config)
print(config)

{'verbose': True}


### Utility functions

In [40]:
def move_to_device(data):
    if isinstance(data, (list,tuple)):
        return [move_to_device(x) for x in data]
    elif isinstance(data, dict):
        return {k: move_to_device(v) for k, v in data.items()}
    elif isinstance(data, torch.Tensor):
        return data.to(DEVICE)
    else:
        return data

## Test the model

In [41]:
def test_model(config, processor, model):
    print("Loading model from checkpoint: ", LAST_CHECKPOINT_NAME)
    
    checkpoint = torch.load(LAST_CHECKPOINT_NAME)
    model.resize_token_embeddings(50344)  ##
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(DEVICE)
    testing_loop(test_dataloader, model, processor, config, f"Test loop")

In [42]:
test_model(config, processor, model)

Loading model from checkpoint:  pix2code_epoch[29]_bleu[0.82].pth


Test loop: 100%|██████████| 18/18 [26:47<00:00, 89.32s/it]


In [44]:
import os
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def read_lines(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f if line.strip()]
    return lines

def compute_bleu(reference_file, prediction_file):
    references = read_lines(reference_file)
    predictions = read_lines(prediction_file)
    
    assert len(references) == len(predictions), f"Line count mismatch: {reference_file} vs {prediction_file}"

    smoothie = SmoothingFunction().method4
    scores = []

    for ref, pred in zip(references, predictions):
        ref_tokens = ref.split()
        pred_tokens = pred.split()
        score = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smoothie)
        scores.append(score)

    return sum(scores) / len(scores)

# Directory path
folder = 'text_pix2code/'
bleu_scores = []

# Scan folder for matching file pairs
for filename in os.listdir(folder):
    if filename.endswith('_answer.txt'):
        base = filename.replace('_answer.txt', '')
        pred_file = os.path.join(folder, f'{base}_pred.txt')
        answer_file = os.path.join(folder, filename)

        if os.path.exists(pred_file):
            try:
                score = compute_bleu(answer_file, pred_file)
                bleu_scores.append((base, score))
                print(f"{base}: BLEU = {score:.4f}")
            except Exception as e:
                print(f"Error processing {base}: {e}")
        else:
            print(f"Missing prediction file for {base}")

# Compute average BLEU
if bleu_scores:
    avg_bleu = sum(score for _, score in bleu_scores) / len(bleu_scores)
    print(f"\nAverage BLEU score across {len(bleu_scores)} files: {avg_bleu:.4f}")
else:
    print("No BLEU scores computed.")


00150311-A7AE-4804-8B7D-9273687B4FC0: BLEU = 0.4937
00CDC9A8-3D73-4291-90EF-49178E408797: BLEU = 0.5550
01B6145F-B88B-4FF8-B7D7-BB8A321EDA85: BLEU = 0.5993
020BDFEE-DA74-4EB0-BF6A-5D29C371983E: BLEU = 0.6383
024A9795-4AEF-4628-B0CD-9B1F1E4FE6C6: BLEU = 0.5760
02B30CB7-A2EE-4483-96BE-22D64E908B76: BLEU = 0.4472
02C4C780-79EA-43F1-B2AE-3A2C757F874C: BLEU = 0.5298
03D5D298-8F87-4DED-AF85-7E7CC2D40924: BLEU = 0.4760
05603517-666B-44EC-86D1-DFB14F81C639: BLEU = 0.4445
06265836-7197-4CF0-9E9F-B7360B20B442: BLEU = 0.4479
076292E2-ABB7-401F-9C93-B72EE17CF54E: BLEU = 0.4973
0831906C-14DB-42DB-A4B3-83947D980165: BLEU = 0.4668
08705928-17E8-4AB8-BF2E-AE9F11BD26A2: BLEU = 0.6047
099F631C-1308-417E-AE1A-4284075AC5D1: BLEU = 0.5433
0B660875-60B4-4E65-9793-3C7EB6C8AFD0: BLEU = 0.5143
0CF69A80-60FE-4B72-9AD3-5DA1C733A492: BLEU = 0.4938
0D1C8ADB-D9F0-48EC-B5AA-205BCF96094E: BLEU = 0.4581
0EBD0467-076F-4946-8549-C3EEF47F37AF: BLEU = 0.3515
0EF165CE-974C-4747-92BB-C16156BF7944: BLEU = 0.4688
1132D1A9-2F7